# Use Case: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) 

In [2]:
!pip install folium

In [2]:
import pandas as pd
import numpy as np
import folium


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv('../../DATA/train_cleaned.csv')

In [4]:
#quick look at the data
train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,is_dropoff_JFK,is_pickup_EWR,is_dropoff_EWR,is_pickup_la_guardia,is_dropoff_la_guardia,trip_distance,pickup_borough,dropoff_borough,is_pickup_lower_manhattan,is_dropoff_lower_manhattan
0,0,0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,...,0,0,0,0,0,0.640487,queens,queens,0,0
1,1,1,1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,...,0,0,0,0,0,5.250670,manhattan,manhattan,1,0
2,2,2,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,...,0,0,0,0,0,0.863411,manhattan,manhattan,0,0
3,3,3,3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,...,0,0,0,0,0,1.739386,manhattan,manhattan,1,0
4,4,4,4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,...,0,0,0,0,0,1.242218,manhattan,manhattan,0,0


In [5]:
train.shape

(400000, 32)

In [6]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

In [7]:
coordinates.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,40.721319,-73.844311,40.712278,-73.841610
1,40.711303,-74.016048,40.782004,-73.979268
2,40.761270,-73.982738,40.750562,-73.991242
3,40.733143,-73.987130,40.758092,-73.991567
4,40.768008,-73.968095,40.783762,-73.956655


## Clustering
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [8]:
from sklearn.cluster import KMeans

In [9]:
#define number of clusters and create instance
k=20
myKMeans=KMeans(n_clusters=k, n_jobs=-1)#parallelize to all cores
#-1 bedeutet benutze alle CPU Cores

In [10]:
#train model
myKMeans.fit(coordinates.to_numpy())

C:\Users\togie\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


KMeans(n_clusters=20, n_jobs=-1)

In [13]:
#get cluster centers
centers=myKMeans.cluster_centers_
labels=myKMeans.labels_    

In [14]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(k):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [15]:
cluster_map

In [28]:
labels

array([ 5, 10,  0, ..., 17,  3,  1])

In [29]:
#add cluster labels to DataFrame
train['clusterID']=labels

In [30]:
#GroupBy Clusters
clusters=train.groupby('clusterID')

In [31]:
clusters['fare_amount'].count()

clusterID
0     76455
1     38895
2      4786
3     40179
4        98
5      3338
6      1706
7      8849
8        53
9      4545
10    33056
11     3127
12     7266
13       93
14    35040
15      726
16    13866
17    55947
18    41319
19    30656
Name: fare_amount, dtype: int64

In [32]:
clusters.mean()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,...,dropoff_longitude_round3,is_pickup_JFK,is_dropoff_JFK,is_pickup_EWR,is_dropoff_EWR,is_pickup_la_guardia,is_dropoff_la_guardia,trip_distance,is_pickup_lower_manhattan,is_dropoff_lower_manhattan
clusterID,,,,,,,,,,,,,,,,,,,,,
0,199613.492891,203938.345367,203938.345367,7.313427,-73.981978,40.757745,-73.981976,40.757837,1.683984,15.679733,...,-73.981974,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.889717,0.025911,0.027716
1,199852.152925,204181.967914,204181.967914,10.528234,-73.979837,40.755972,-73.957930,40.779361,1.671757,15.647641,...,-73.957933,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.155009,0.128217,0.000000
2,201579.652946,205947.030297,205947.030297,48.660740,-73.784753,40.646472,-73.971099,40.739415,1.800460,15.727121,...,-73.971107,0.976181,0.000000,0.000000,0.001045,0.000000,0.011910,12.109914,0.000000,0.176557
3,200683.615371,205031.394086,205031.394086,8.229309,-73.998472,40.723352,-73.999372,40.723095,1.679584,15.715697,...,-73.999369,0.000000,0.000000,0.000025,0.000000,0.000000,0.000000,1.175396,0.925434,0.951393
4,208037.275510,212544.540816,212544.540816,9.795102,-73.149918,41.366595,-73.147641,41.368939,1.387755,16.173469,...,-73.147255,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.242551,0.000000,0.000000
5,203191.077292,207593.214500,207593.214500,12.670839,-73.885575,40.762016,-73.881417,40.755101,1.711204,15.727681,...,-73.881406,0.000000,0.001498,0.000000,0.000000,0.343619,0.121630,1.884286,0.000000,0.000000
6,201137.485932,205494.896835,205494.896835,23.598224,-73.787808,40.655775,-73.796676,40.688269,1.677608,15.679953,...,-73.796655,0.821805,0.332943,0.000000,0.000000,0.000000,0.019930,3.346593,0.000000,0.000000
7,200940.021923,205293.528421,205293.528421,27.805218,-73.980338,40.751899,-73.876804,40.761520,1.687761,15.715900,...,-73.876808,0.000000,0.000000,0.000000,0.000000,0.000000,0.549667,5.775098,0.256300,0.000000
8,188799.094340,192893.528302,192893.528302,14.055283,-73.150355,41.359023,-73.968754,40.746737,1.377358,15.415094,...,-73.968698,0.000000,0.018868,0.000000,0.000000,0.000000,0.056604,60.241047,0.000000,0.377358


In [33]:
clusters.var()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,...,dropoff_longitude_round3,is_pickup_JFK,is_dropoff_JFK,is_pickup_EWR,is_dropoff_EWR,is_pickup_la_guardia,is_dropoff_la_guardia,trip_distance,is_pickup_lower_manhattan,is_dropoff_lower_manhattan
clusterID,,,,,,,,,,,,,,,,,,,,,
0,1.338339e+10,1.396506e+10,1.396506e+10,14.632984,0.000069,0.000072,0.000077,0.000076,1.708109,75.207078,...,0.000077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.195089,0.025240,0.026948
1,1.333075e+10,1.391004e+10,1.391004e+10,23.341573,0.000114,0.000133,0.000131,0.000167,1.698677,74.918237,...,0.000131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.519251,0.111780,0.000000
2,1.366069e+10,1.425459e+10,1.425459e+10,131.052168,0.000127,0.000064,0.001003,0.002172,1.903122,76.580067,...,0.001003,0.023257,0.000000,0.000000,0.001044,0.000000,0.011770,3.431476,0.000000,0.145415
3,1.329890e+10,1.387688e+10,1.387688e+10,24.086272,0.000126,0.000147,0.000109,0.000119,1.672083,75.381687,...,0.000109,0.000000,0.000000,0.000025,0.000000,0.000000,0.000000,0.540363,0.069008,0.046246
4,1.368280e+10,1.427686e+10,1.427686e+10,54.881506,0.005296,0.000353,0.004019,0.000394,0.610983,91.217021,...,0.004026,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.412272,0.000000,0.000000
5,1.349417e+10,1.408068e+10,1.408068e+10,191.419415,0.000844,0.000575,0.001425,0.000995,1.820078,76.441553,...,0.001424,0.000000,0.001496,0.000000,0.000000,0.225613,0.106868,3.615568,0.000000,0.000000
6,1.336302e+10,1.394363e+10,1.394363e+10,362.772861,0.000505,0.000672,0.002209,0.002335,1.732953,75.965545,...,0.002210,0.146527,0.222222,0.000000,0.000000,0.000000,0.019544,12.014641,0.000000,0.000000
7,1.345088e+10,1.403553e+10,1.403553e+10,80.410189,0.000255,0.000488,0.000541,0.000382,1.691036,74.675155,...,0.000540,0.000000,0.000000,0.000000,0.000000,0.000000,0.247561,2.565978,0.190632,0.000000
8,1.584704e+10,1.653706e+10,1.653706e+10,211.167945,0.004627,0.002996,0.002470,0.000999,0.739478,67.785922,...,0.002463,0.000000,0.018868,0.000000,0.000000,0.000000,0.054427,15.865221,0.000000,0.239478
